In [ ]:
#https://github.com/kenwaytis/faster-SadTalker-API/blob/main/main.py

!python -m pip install elevenlabs
!python -m pip install openai
!python -m pip install numpy
!python -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!python -m pip install moviepy
!python -m pip install imageio==2.4.1
!python -m pip install -r requirements.txt
!python -m pip install cmake==3.25.2
!python -m pip install boost==0.1
!python -m pip install dlib-bin
!python -m pip install basicsr==1.4.2
!python -m pip install facexlib==0.2.5
!python -m pip install kornia==0.6.10
!python -m pip install face-alignment==1.3.4 yacs==0.1.8

### install gpfgan for enhancer
!python -m pip install git+https://github.com/TencentARC/GFPGAN

In [20]:
import json
import elevenlabs
from openai import OpenAI
import time
import uuid
from dotenv import main
import matplotlib.pyplot as plt
from PIL import Image
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip
from openai import OpenAI

####
import glob
import numpy
import subprocess
import sys
import os
import requests

In [22]:
main.load_dotenv()
openai_api_key = os.getenv('openai_api_key')
elevenlabs_api_key = os.getenv("elevenlabs_api_key")

#connect to elevenlabs, define the model 
elevenlabs.set_api_key(elevenlabs_api_key)
voice_id = "vDxT1hhO5CjgzUI7e2vF"

#connect to specific assistant and open new thread
client = OpenAI(api_key=openai_api_key)
assistant_id = "asst_nv9sFR2LXHefrpwws4EUT6xU"
my_assistant = client.beta.assistants.retrieve(assistant_id)
new_thread = client.beta.threads.create()

In [23]:
def video_generation(audio_path, audio_id):
    
    !python ./SadTalker/inference.py \
      --driven_audio test.wav \
      --source_image ./SadTalker/example_img.jpg \
      --result_dir ./results/

In [24]:
def new_question(question_text):
  message_request = client.beta.threads.messages.create(
      thread_id = new_thread.id,
      role = "user",
      content = "In 30 words or less (important). " + question_text
  )

  run = client.beta.threads.runs.create(
    thread_id = message_request.thread_id,
    assistant_id = my_assistant.id
  )

  run = client.beta.threads.runs.retrieve(
    thread_id = message_request.thread_id,
    run_id = run.id
  )

  def wait_for_completed():
      print('thread message sent')
      while True:
          response = client.beta.threads.runs.retrieve(thread_id = new_thread.id, run_id = run.id)
          if response.status == "completed":
              print('thread response received')
              return response
          time.sleep(1)

  run_return = wait_for_completed()

  messages = client.beta.threads.messages.list(
    thread_id = message_request.thread_id
  )

  audio = elevenlabs.generate(
      text = messages.data[0].content[0].text.value,
          voice = voice_id
  )

  audio_id = uuid.uuid4()
  elevenlabs.save(audio, "test.wav")
  video_generation("test.wav", audio_id)

In [25]:
new_question("What is the best morning routine?")

thread message sent


KeyboardInterrupt: 